In [1]:
#####################################################################################
# LDA_topic_distribution_parser.py
#
# Description:
# This script takes the LDA output file and
# parses it to separate the topic distribution scores into separate columns.
#
# Version: 0.3
# Date Created: 09/03/2019
# Last Modified: 07/16/2020
#####################################################################################

In [2]:
### GLOBAL VARIABLES ###

PROJECT_FOLDER = "C:/Users/kathleen.trinh/Documents/Nike/Women's Plus Size/LDA Reddit/"
FILENAME = "Nike_Women's-Plus-Size_Reddit_ALL_CLEAN_LDA_output"

# Specify the number of topics previously used for the LDA process
NUM_TOPICS = 12

In [3]:
### IMPORTS ###

import copy
import numpy as np
import os
import pandas as pd
import re
import string

In [4]:
### SET-UP ###

# Load .csv file and put data in dataframe
df = pd.read_csv((PROJECT_FOLDER + FILENAME + ".csv"), header=0, encoding='utf-8')

# Rename columns
#df.columns = ["My Comment ID", "Clean Body no stopwords"]

# Print the dataframe's dimensions
print("Dataframe Dimensions: {} Rows, {} Columns".format(*df.shape))

# Sample the df's data
df.sample(5)

Dataframe Dimensions: 326096 Rows, 11 Columns


,my_comment_id,subreddit,title,body,permalink,created,timestamp,clean_body,clean_body_no_stopwords,BoW_Representation,Topic_Distribution
239603,Reddit_t3_dobcbx_f5py2gm_Com11,ABraThatFits,"I went to a lingerie store, called 'your perfe...",JCpenny should also have at least a 42DDD in s...,/r/ABraThatFits/comments/dobcbx/i_went_to_a_li...,1.572399e+09,2019-10-29 18:21:42,jcpenny should also have at least a 42ddd in s...,jcpenny least 42ddd store starting point,"[(42, 1), (246, 1), (667, 1), (4771, 1), (9370...","[(0, 0.016671786), (1, 0.016670998), (2, 0.016..."
306934,Reddit_t3_gox7bh_friivc2_Com2,ABraThatFits,"[Recommendations] Full support, side support, ...","Hello, thank you for submitting a post to /r/A...",/r/ABraThatFits/comments/gox7bh/recommendation...,1.590230e+09,2020-05-23 03:38:57,"hello, thank you for submitting a post to /r/a...",submit abrathatfits feel safe posting never se...,"[(193, 1), (236, 1), (344, 1), (467, 1), (949,...","[(0, 0.060673486), (2, 0.13947898), (5, 0.0302..."
251591,Reddit_t3_eldmgd_fdh9mtz_Com8,bigboobproblems,Workout class instructors don't understand tha...,"If it's a yoga place steal blocks for plank, i...",/r/bigboobproblems/comments/eldmgd/workout_cla...,1.578447e+09,2020-01-07 17:32:46,"if it's a yoga place steal blocks for plank, i...",yoga place steal block plank shifts little wei...,"[(193, 1), (243, 1), (355, 1), (379, 1), (513,...","[(1, 0.2411778), (3, 0.08665931), (5, 0.078507..."
91414,Reddit_t3_4s8cu9_d57h0j5_Com9,Fitness,Is it better to squat every day or every other...,It's amazing how when you do a lot more of som...,/r/Fitness/comments/4s8cu9/is_it_better_to_squ...,1.468238e+09,2016-07-11 04:56:25,it's amazing how when you do a lot more of som...,amazing big result,"[(27, 1), (360, 1), (1775, 1)]","[(0, 0.24092172), (1, 0.020835578), (2, 0.0208..."
82606,Reddit_t3_4c6smp_d1fucff_Com5,xxfitness,I have quads and a limited budget. How do I pa...,I've had really good luck with the curvy fit a...,/r/xxfitness/comments/4c6smp/i_have_quads_and_...,1.459157e+09,2016-03-28 02:19:02,i've had really good luck with the curvy fit a...,luck curvy fit old navy,"[(39, 1), (413, 1), (676, 1), (1637, 1), (2198...","[(0, 0.013890109), (1, 0.013889693), (2, 0.013..."


In [7]:
### FUNCTION TO MAKE topic_dist_lists FROM LDA OUTPUT FILE'S DF ###

def make_topic_dist_lists_from_df(df):
    topic_dist_lists = []
    raw_topic_dist_lists = []
    for row in df["Topic_Distribution"]:
        split_row = row.split("[]()")
        raw_topic_dist_lists.append(split_row)
    for r_TD_list in raw_topic_dist_lists:
        TD_list = []
        no_brackets_string = r_TD_list[0].strip("[]")
        string_list = no_brackets_string.split("), ")
        string_list = [tuple_string.strip("()") for tuple_string in string_list]
        for tuple_string in string_list:
            topic_num = int(tuple_string.split(", ")[0])
            topic_prob = float(tuple_string.split(", ")[1])
            TD_list.append((topic_num, topic_prob))
        topic_dist_lists.append(TD_list)
    return topic_dist_lists

### PARSE EACH TOPIC DISTRIBUTION LIST FOR EACH DOC ### (vvv FUNCTIONS FROM OVERALL LDA SCRIPT vvv)

def init_TopicProbLists(num_docs):
    init_list = [0 for x in range(0, num_docs)]
    topic_lists = [copy.deepcopy(init_list) for x in range(0, NUM_TOPICS)]
    return topic_lists

def parse_TD_list(topic_lists, doc_counter, TD_list):
    for topic_prob_pair in TD_list:
        topic_num = topic_prob_pair[0]
        probability = topic_prob_pair[1]
        #print("Topic #" + str(topic_num) + " probability = " + str(probability))
        #print("\t Accessing... topic_lists[" + str(topic_num) + "][" + str(doc_counter) + "]")
        topic_lists[topic_num][doc_counter] = probability
        #for T_list in topic_lists:
        #    print(T_list)
    return topic_lists
        
def parse_TopicDistLists(topic_dist_lists, topic_lists):
    print("Parsing ", end = "")
    doc_counter = 0
    for TD_list in topic_dist_lists:
        #print((20 * "-"), ("Doc #" + str(doc_counter)), (20 * "-"))
        topic_lists = parse_TD_list(topic_lists, doc_counter, TD_list)
        doc_counter += 1
        #print()
        print(". ", end = "")
    print()
    return topic_lists

In [ ]:
topic_dist_lists = make_topic_dist_lists_from_df(df)
print("Length of topic_dist_lists:", len(topic_dist_lists))

In [8]:
print("Testing functions with 10 lists...")
topic_lists = init_TopicProbLists(10)
topic_lists = parse_TopicDistLists(topic_dist_lists[0:10], topic_lists)

print((20 * "-"), "END RESULT", (20 * "-"))
for T_list in topic_lists:
    print(T_list)

Testing functions with 10 lists...
Parsing . . . . . . . . . . 
-------------------- END RESULT --------------------
[0.016667305, 0.01195454, 0.07822738, 0.02242796, 0.010426058, 0, 0, 0.18473054, 0, 0.15679897]
[0.016667658, 0.01195384, 0, 0.26121938, 0.35954764, 0.110439144, 0.10915855, 0, 0, 0.010417628]
[0.016667305, 0.011953723, 0.14329164, 0, 0.010426058, 0.08027245, 0, 0, 0.09873605, 0.010417893]
[0.016667305, 0.011954578, 0, 0, 0.23718019, 0, 0, 0, 0, 0.010417718]
[0.8166567, 0.38514006, 0.2313924, 0.32256594, 0.18464512, 0, 0.2540462, 0.34898233, 0.73414594, 0.0104181105]
[0.016667448, 0.09242293, 0, 0.032835674, 0.010426058, 0, 0.21522091, 0, 0, 0.010418944]
[0.016667435, 0.20168896, 0, 0, 0.13521817, 0, 0.03579362, 0, 0.0987418, 0.010417613]
[0.016668553, 0.011953923, 0.33852214, 0.24972802, 0.010426429, 0.35818648, 0.23652893, 0.3204489, 0, 0.7390224]
[0.016668007, 0.011953894, 0.1720679, 0.046030436, 0.010426095, 0, 0, 0, 0, 0.010417665]
[0.016667305, 0.011953723, 0, 0.05

In [9]:
print("Processing all lists in topic_dist_lists (", len(topic_dist_lists), "items )...")

topic_lists = init_TopicProbLists(len(topic_dist_lists))
topic_lists = parse_TopicDistLists(topic_dist_lists, topic_lists)

Processing all lists in topic_dist_lists ( 326096 items )...


Parsing . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

...

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [14]:
# Print the first 10 values of each topic list to see if data populated
i = 1
for T_list in topic_lists:
    print("Topic #" + str(i) + ": " + str(T_list[0:10]))
    i += 1

Topic #1: [0.016667305, 0.01195454, 0.07822738, 0.02242796, 0.010426058, 0, 0, 0.18473054, 0, 0.15679897]
Topic #2: [0.016667658, 0.01195384, 0, 0.26121938, 0.35954764, 0.110439144, 0.10915855, 0, 0, 0.010417628]
Topic #3: [0.016667305, 0.011953723, 0.14329164, 0, 0.010426058, 0.08027245, 0, 0, 0.09873605, 0.010417893]
Topic #4: [0.016667305, 0.011954578, 0, 0, 0.23718019, 0, 0, 0, 0, 0.010417718]
Topic #5: [0.8166567, 0.38514006, 0.2313924, 0.32256594, 0.18464512, 0, 0.2540462, 0.34898233, 0.73414594, 0.0104181105]
Topic #6: [0.016667448, 0.09242293, 0, 0.032835674, 0.010426058, 0, 0.21522091, 0, 0, 0.010418944]
Topic #7: [0.016667435, 0.20168896, 0, 0, 0.13521817, 0, 0.03579362, 0, 0.0987418, 0.010417613]
Topic #8: [0.016668553, 0.011953923, 0.33852214, 0.24972802, 0.010426429, 0.35818648, 0.23652893, 0.3204489, 0, 0.7390224]
Topic #9: [0.016668007, 0.011953894, 0.1720679, 0.046030436, 0.010426095, 0, 0, 0, 0, 0.010417665]
Topic #10: [0.016667305, 0.011953723, 0, 0.05906284, 0.010426

In [15]:
for i in range(0, NUM_TOPICS):
    for j in range(8400, 8430):
        print(topic_lists[i][j], ", ", end = "")
    print()

0.016678056 , 0.041668262 , 0.016669508 , 0.027777897 , 0.010426209 , 0.1546385 , 0.013889864 , 0.016671889 , 0.013915669 , 0.34726256 , 0 , 0 , 0.011905446 , 0 , 0 , 0.02778015 , 0.016668655 , 0.17144309 , 0.027780768 , 0.041666247 , 0.013895743 , 0.027778236 , 0 , 0.010421002 , 0.013892629 , 0.011906225 , 0 , 0.027779665 , 0.013894153 , 0.020834532 , 
0.21626537 , 0.04166898 , 0.0166696 , 0.02777803 , 0.010426152 , 0.011906612 , 0.013890508 , 0.016671725 , 0.1802671 , 0.013891288 , 0 , 0 , 0.011905446 , 0.28193316 , 0 , 0.02778015 , 0.01666872 , 0 , 0.027782472 , 0.041666247 , 0.18030845 , 0.027778236 , 0.13289145 , 0.01042104 , 0.013892629 , 0.20882201 , 0 , 0.027781917 , 0.013891846 , 0.020835232 , 
0.016678056 , 0.041668262 , 0.016669508 , 0.027780352 , 0.37171987 , 0.7262923 , 0.25428465 , 0.21670407 , 0.1808934 , 0.013891288 , 0.08742922 , 0.3497361 , 0.011906306 , 0 , 0 , 0.02778015 , 0.016668655 , 0 , 0.027780768 , 0.041666247 , 0.013895938 , 0.027778236 , 0.07323813 , 0.01042

In [ ]:
# Reload .csv file and put data in dataframe
#output_df = pd.read_csv((PROJECT_FOLDER + FILENAME + ".csv"), header=0, encoding='utf-8')

In [16]:
df = df.assign(Topic_1 = topic_lists[0])
df = df.assign(Topic_2 = topic_lists[1])
df = df.assign(Topic_3 = topic_lists[2])
df = df.assign(Topic_4 = topic_lists[3])
df = df.assign(Topic_5 = topic_lists[4])
df = df.assign(Topic_6 = topic_lists[5])
df = df.assign(Topic_7 = topic_lists[6])
df = df.assign(Topic_8 = topic_lists[7])
df = df.assign(Topic_9 = topic_lists[8])
df = df.assign(Topic_10 = topic_lists[9])
df = df.assign(Topic_11 = topic_lists[10])
df = df.assign(Topic_12 = topic_lists[11])
#output_df = output_df.assign(Topic_13 = topic_lists[12])
#output_df = output_df.assign(Topic_14 = topic_lists[13])
#output_df = output_df.assign(Topic_15 = topic_lists[14])
#output_df = output_df.assign(Topic_16 = topic_lists[15])
#output_df = output_df.assign(Topic_17 = topic_lists[16])
#output_df = output_df.assign(Topic_18 = topic_lists[17])
#output_df = output_df.assign(Topic_19 = topic_lists[18])
#output_df = output_df.assign(Topic_20 = topic_lists[19])

In [17]:
df.sample(5)

,my_comment_id,subreddit,title,body,permalink,created,timestamp,clean_body,clean_body_no_stopwords,BoW_Representation,...,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Topic_10,Topic_11,Topic_12
227497,Reddit_t3_crlceu_ex7aa3z_Com226,fatlogic,"Found on Facebook, taken in a vintage store","This always bums me out. I'm tall, and I have ...",/r/fatlogic/comments/crlceu/found_on_facebook_...,1.566087e+09,2019-08-17 17:06:42,"this always bums me out. i'm tall, and i have ...",bum tall broad shoulder long arm nothing vinta...,"[(39, 1), (52, 1), (75, 1), (84, 1), (189, 1),...",...,0.000000,0.000000,0.478992,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.189349
224411,Reddit_t3_ci7k7j_ev28efr_Com1,PlusSize,Terrified of Flying,I flew on both of these types of planes when I...,/r/PlusSize/comments/ci7k7j/terrified_of_flyin...,1.564201e+09,2019-07-26 21:18:11,i flew on both of these types of planes when i...,flew type plane 5 month pregnant similar start...,"[(39, 1), (65, 1), (73, 1), (134, 1), (155, 1)...",...,0.063545,0.124743,0.110950,0.000000,0.060659,0.121510,0.000000,0.057374,0.0,0.167540
152374,Reddit_t3_80d43c_duv0ujz_Com8,fatlogic,"""Body positivity could be one of the most impo...",My take on Body positivity:\r\n\r\nHow it shou...,/r/fatlogic/comments/80d43c/body_positivity_co...,1.519697e+09,2018-02-26 17:55:42,my take on body positivity:\r\n\r\nhow it shou...,body positivity teach disfigure scar disabled ...,"[(19, 1), (65, 1), (232, 2), (337, 1), (857, 1...",...,0.000000,0.079813,0.000000,0.000000,0.000000,0.000000,0.071169,0.000000,0.0,0.381078
96742,Reddit_t3_53pvd6_d7wm6z1_Com128,fatlogic,Creating a 300 calories deficit isn't harder f...,That's why I also exercise anyway. \r\n\r\nAnd...,/r/fatlogic/comments/53pvd6/creating_a_300_cal...,1.474523e+09,2016-09-21 22:35:47,that's why i also exercise anyway. \r\n\r\nand...,exercise 20 lb lose instead 60 taller bmi ever...,"[(122, 1), (247, 1), (348, 1), (392, 1), (405,...",...,0.296140,0.496387,0.000000,0.000000,0.000000,0.000000,0.139282,0.000000,0.0,0.000000
72919,Reddit_t3_3qgvfx_cwf6m2j_Com2,ABraThatFits,[Update][Discussion] My employer hosted a bra-...,Awesome! (Though I do want to point out bras m...,/r/ABraThatFits/comments/3qgvfx/updatediscussi...,1.446015e+09,2015-10-27 23:46:50,awesome! (though i do want to point out bras m...,awesome point bra interfere lymph lead cancer ...,"[(29, 2), (239, 1), (410, 1), (433, 1), (667, ...",...,0.000000,0.609238,0.068437,0.083351,0.000000,0.187673,0.000000,0.000000,0.0,0.000000


In [18]:
df.to_csv((PROJECT_FOLDER + FILENAME + "_LDA_output_parsed.csv"), index=None, header=True, encoding='utf-8')